# Chapter 3 Exercises

## Exercise 3.1: Hand calculations for the trapezoidal method

Hand computations:
    $$ \int_1^3 2x^3 dx \approx \frac{f(1)+f(3)}{2} + f(2) = \frac{2+54}{2} + 16 = 44 $$

In [15]:
# trapezoidal_test_func.py
from trapezoidal import trapezoidal

def test_trapezoidal():
    f = lambda x: 2*x**3
    x0, x1, n = 1, 3, 2
    
    # Comparer hand computations with system output
    I_hand = 44
    I_sys = trapezoidal(f, x0, x1, n)
    error = abs(I_hand - I_sys)
    tol = 1e-16
    assert error < tol
    
test_trapezoidal()

## Exercise 3.2: Hand calculations for the midpoint method

Hand computations:
    $$ \int_1^3 2x^3 dx \approx f\left(\frac32\right) + f\left(\frac52\right) = 2 \left(\frac32\right)^3 + 2 \left(\frac52\right)^3 = 38 $$

In [14]:
# midpoint_test_func.py
from midpoint import midpoint

def test_midpoint():
    f = lambda x: 2*x**3
    x0, x1, n = 1, 3, 2
    
    # Comparer hand computations with system output
    I_hand = 38
    I_sys = midpoint(f, x0, x1, n)
    error = abs(I_hand - I_sys)
    tol = 1e-16
    assert error < tol
    
test_midpoint()

## Exercise 3.3: Compute a simple integral

Hand computations for approximations of $\int_2^6 x(x-1)dx$.

\begin{align}
  n &= 2, I_t = \left(\frac{f(2)+f(6)}{2} + f(4)\right) \left(\frac{6-2}{2}\right)
  = \left(\frac{2(2-1)+6(6-1)}{2} + 4(4-1)\right) (2) = 56 \\
  n &= 2, I_m = \left( f(3) + f(5) \right) \left(\frac{6-2}{2}\right)
  = (3(3-1) + 5(5-1))(2) = 52
\end{align}

Not needed, misread question.

In [25]:
# integrate_parabola.py
from trapezoidal import trapezoidal
from midpoint import midpoint
import sympy as sp

def test_methods():
    f = lambda x: x*(x-1)
    x0, x1 = 2, 6

    # Exact integral by sympy
    x = sp.symbols('x')
    I_exact = sp.integrate(f(x), (x, x0, x1))
    I_exact = I_exact.evalf()

    # numerical integration
    nvec = [2, 100]
    print "  n  trapezoidal       error |   midpoint       error"
    for n in nvec:
        I_t = trapezoidal(f, x0, x1, n)
        e_t = abs(I_t - I_exact)
        I_m = midpoint(f, x0, x1, n)
        e_m = abs(I_t - I_exact)
        print("%3i   %10.6f  %10.6f | %10.6f  %10.6f" % (n, I_t, e_t, I_m, e_m))
        
test_methods()

  n  trapezoidal       error |   midpoint       error
  2    56.000000    2.666667 |  52.000000    2.666667
100    53.334400    0.001067 |  53.332800    0.001067


## Exercise 3.4: Hand-calculations with sine integrals

$\int_0^\pi \sin(x) dx = -\cos(\pi) -(-\cos(0)) = 2$

In [37]:
from trapezoidal import trapezoidal
from midpoint import midpoint
from math import pi, sin

def test_integrate_sine():
    f = lambda x: sin(x)
    x0, x1, n = 0, pi, int(1e2)
    exact = 2
    numer_t = trapezoidal(f, x0, x1, n)
    numer_m = midpoint(f, x0, x1, n)
    err_t = abs(exact - numer_t)
    err_m = abs(exact - numer_m)
    tol = 1e-6
    assert err_t < tol, "err_t = %g" % (err_t)
    assert err_m < tol, "err_m = %g" % (err_m)

test_integrate_sine()

AssertionError: err_t = 0.000164496

It converges rather slow.  Large `n` and `tol` are needed to avoid `AssertionError`.